In [1]:
import sys
import os

# Uzyskaj bieżący katalog roboczy
current_dir = os.getcwd()  # Uzyskaj bieżący katalog roboczy
src_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Ścieżka do katalogu src

# Dodaj katalog src do sys.path
sys.path.append(src_dir)


In [2]:
import logging
logging.basicConfig(level=logging.INFO, filename='flow.log')


In [3]:
from src.flow.PubmedEndpoint import PubmedEndpoint

publication_text = PubmedEndpoint.fetch_full_text_from_pubmed_id('21441262')

'<?xml version="1.0" ?>\n<!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd">\n<pmc-articleset><article xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:mml="http://www.w3.org/1998/Math/MathML" article-type="research-article">\n  <!--The publisher of this article does not allow downloading of the full text in XML form.-->\n  <?properties manuscript?>\n  <front>\n    <journal-meta>\n      <journal-id journal-id-type="nlm-journal-id">2985087R</journal-id>\n      <journal-id journal-id-type="pubmed-jr-id">4945</journal-id>\n      <journal-id journal-id-type="nlm-ta">J Med Genet</journal-id>\n      <journal-id journal-id-type="iso-abbrev">J. Med. Genet.</journal-id>\n      <journal-title-group>\n        <journal-title>Journal of medical genetics</journal-title>\n      </journal-title-group>\n      <issn pub-type="ppub">0022-2593</issn>\n      <issn pub-type="epub">1468-6244</issn>\n    </journal-meta>\n    <

In [11]:
with open('../data/pubmed-articles/21441262.txt', 'r') as file:
    publication_text = file.read()

In [10]:
from src.LlmManager import LlmManager
llm_manager = LlmManager('gpt', 'gpt-4o-mini')

In [14]:
import importlib
import src.flow.coordinates_extraction.CoordinatesRegexExtractor
importlib.reload(src.flow.coordinates_extraction.CoordinatesRegexExtractor) 
from src.flow.coordinates_extraction.CoordinatesRegexExtractor import CoordinatesRegexExtractor

import src.flow.coordinates_extraction.CoordinatesLlmExtractor
importlib.reload(src.flow.coordinates_extraction.CoordinatesLlmExtractor) 
from src.flow.coordinates_extraction.CoordinatesLlmExtractor import CoordinatesLlmExtractor

import src.LlmManager
importlib.reload(src.LlmManager) 
from src.LlmManager import LlmManager



coordinates_regex_extractor = CoordinatesRegexExtractor()
regex_extracted_coordinates = coordinates_regex_extractor.extract_coordinates(publication_text)

llm_manager = LlmManager('gpt', 'gpt-4o-mini')
llm = llm_manager.get_llm()

coordinates_llm_extractor = CoordinatesLlmExtractor(llm)
llm_extracted_coordinates = coordinates_llm_extractor.extract(publication_text)
print("Regex extracted coordinates: ",regex_extracted_coordinates)
print("LLM extracted coordinates: ",llm_extracted_coordinates)

Regex extracted coordinates:  []
LLM extracted coordinates:  ['NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', '', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695', 'NC_000014.8:g.101179660_101179695']


In [11]:
import json
with open('../data/pubmed-articles/21441262.json', 'r') as file:
    y = json.load(file)

In [17]:
y_coordinates = list(map(lambda x: x['coordinate'], next(article for article in y if article['pubmed_id'] == '21441262')['coordinates']))
y_coordinates


['46,XX,t(2;14)(q13;q13)',
 '14q12-q13.1',
 '14q12',
 'chr14:28,754,647-28,756,495 bp',
 '28,930,280-28,932,099 bp',
 '~10 kb gene-poor region in 14q12 located ~265 kb downstream of FOXG1',
 '1-Mb region downstream of FOXG1',
 'chr14:28,930,280-28,932,099',
 '46,XX,del(14)(q13.1q13.2)',
 '~5.4-Mb heterozygous deletion in 14q12-q13.1',
 '~1.09-Mb microdeletion downstream of FOXG1',
 'transitions c.757A>G (p.N253D)',
 'c.700T>C (p.S234P)',
 'c.460dupG (p.E154GfsX301)',
 'c.256C>T (p.R86X)',
 'c.263_278del16 (p.R88PfsX99)',
 'c.460dupG mutation']

In [3]:
from src.ContextExtractionService import ContextExtractionService

context_extraction_service = ContextExtractionService('gpt-4o-mini')
coordinate = coordinates_list[3]
coordinate_context = context_extraction_service.extract_context_from_coordinate(publication_text, coordinate)
print(coordinate_context)

The following sentences from the publication are relevant to the genomic coordinates chr10:23508365A>G and chr10:23508446A>C:

1. "We also identified a homozygous mutation in one patient with pancreatic agenesis and intrahepatic cholestatic failure."
2. "Of the 10 probands with mutations in this element, 6 had the same chr10:23508437A>G mutation as part of a shared extended haplotype (minimal shared haplotype of 1.2Mb)."
3. "Three of the remaining probands had different base substitution mutations: a homozygous chr10:23508363A>G mutation, a homozygous chr10:23508305A>G mutation and compound heterozygous chr10:23508365A>G/chr10:23508446A>C mutations."
4. "Testing of parents and siblings demonstrated co-segregation of the mutations with diabetes and exocrine insufficiency."

These sentences provide context about the specific mutations at the given coordinates, their occurrence in patients with pancreatic agenesis, and the analysis of genetic inheritance related to these mutations.


In [5]:
import importlib
import src.SequenceOntologyMappingService  # Initial import
importlib.reload(src.SequenceOntologyMappingService)  # Reload the module
from src.SequenceOntologyMappingService import SequenceOntologyMappingService  # Re-import the class

sequence_ontology_mapping_service = SequenceOntologyMappingService('gpt-4o-mini')
sequence_ontology_term = sequence_ontology_mapping_service.map_coordinates_to_sequence_ontology(coordinate, coordinate_context)
print(sequence_ontology_term)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


```json
{"name": "minus_1_frameshift_variant", "definition": "\"A sequence variant which causes a disruption of the translational reading frame, by shifting one base ahead.\" [http://arjournals.annualreviews.org/doi/pdf/10.1146/annurev.ge.08.120174.001535]", "comment": null, "_distance": 156.83042907714844}
```


In [17]:
import src.LinksFromQueryExtractionService
importlib.reload(src.LinksFromQueryExtractionService)
from src.LinksFromQueryExtractionService import LinksFromQueryExtractionService

links_from_query_extraction_service = LinksFromQueryExtractionService('gpt-4o-mini')
user_query_dict = {'type of variant': 'frameshift'}
links = links_from_query_extraction_service.extract_links_from_query(coordinate, coordinate_context, sequence_ontology_term, user_query_dict)

Link between type of variant and frameshift: False. Type of relation is RELATION_TYPE  . Reasoning: The context indicates that the mutations listed (including chr10:23508365A>G and chr10:23508446A>C) do not specifically refer to a frameshift variant. Instead, the provided sequence ontology describes a "minus_1_frameshift_variant," which is a specific type of sequence variant that disrupts the translational reading frame. There is no direct evidence in the context provided that links the user value "frameshift" specifically to the mentioned coordinates as a confirmed type of variant.


In [10]:
coordinate_context

'The following sentences from the publication are relevant to the genomic coordinates chr10:23508365A>G and chr10:23508446A>C:\n\n1. "We also identified a homozygous mutation in one patient with pancreatic agenesis and intrahepatic cholestatic failure."\n2. "Of the 10 probands with mutations in this element, 6 had the same chr10:23508437A>G mutation as part of a shared extended haplotype (minimal shared haplotype of 1.2Mb)."\n3. "Three of the remaining probands had different base substitution mutations: a homozygous chr10:23508363A>G mutation, a homozygous chr10:23508305A>G mutation and compound heterozygous chr10:23508365A>G/chr10:23508446A>C mutations."\n4. "Testing of parents and siblings demonstrated co-segregation of the mutations with diabetes and exocrine insufficiency."\n\nThese sentences provide context about the specific mutations at the given coordinates, their occurrence in patients with pancreatic agenesis, and the analysis of genetic inheritance related to these mutations